<a href="https://colab.research.google.com/github/dessqa/dessqa/blob/main/kazan_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange 
import json
import re

In [18]:
test_data = pd.read_parquet('/content/sample_data/test.parquet')

In [19]:
test_data.head()

,product_id,sale,shop_id,shop_title,rating,text_fields
1,1997646,False,22758,Sky_Electronics,5.000000,"{""title"": ""Светодиодная лента Smart led Strip ..."
2,927375,False,17729,Di-Di Market,4.405941,"{""title"": ""Стекло ПЛЕНКА керамик матовое Honor..."
3,1921513,False,54327,VisionStore,4.000000,"{""title"": ""Проводные наушники с микрофоном jac..."
4,1668662,False,15000,FORNAILS,5.000000,"{""title"": ""Декоративная табличка \""Правила кух..."
5,1467778,False,39600,МОЯ КУХНЯ,5.000000,"{""title"": ""Подставка под ложку керамическая, п..."


In [20]:
train_data = pd.read_parquet('/content/sample_data/train.parquet')

In [21]:
train_data.head()

,product_id,category_id,sale,shop_id,shop_title,rating,text_fields,category_name
0,325286,12171,False,9031,Aksik,5.0,"{""title"": ""Зарядный кабель Borofone BX1 Lightn...",Все категории->Электроника->Смартфоны и телефо...
1,888134,14233,False,18305,Sela,5.0,"{""title"": ""Трусы Sela"", ""description"": ""Трусы-...",Все категории->Одежда->Женская одежда->Белье и...
3,1267173,13429,False,16357,ЮНЛАНДИЯ канцтовары,5.0,"{""title"": ""Гуашь \""ЮНЫЙ ВОЛШЕБНИК\"", 12 цветов...",Все категории->Хобби и творчество->Рисование->...
4,1416943,2789,False,34666,вася-nicotine,4.0,"{""title"": ""Колба для кальяна Крафт (разные цве...",Все категории->Хобби и творчество->Товары для ...
5,1058275,12834,False,26389,Lim Market,4.6,"{""title"": ""Пижама женская, однотонная с шортам...",Все категории->Одежда->Женская одежда->Домашня...


In [22]:
# Функция пробразования столбца text_fields в столбцы трейна
def str_to_df(df):
    var = df.text_fields                                                    # Нужный столбец
    var = var.apply(lambda x: json.loads(x))                                # Прогоняем через json.loads
    var = var.apply(pd.Series)                                              # Преобразуем в серию
    final_df = pd.concat([df,var],axis=1).drop('text_fields',axis=1)        # Склеиваем
    return final_df

In [23]:
train_data.shape

(91120, 8)

In [24]:
test_data.shape

(16860, 6)

In [25]:
final_train_data = str_to_df(train_data)

In [26]:
final_train_data.head()

,product_id,category_id,sale,shop_id,shop_title,rating,category_name,title,description,attributes,custom_characteristics,defined_characteristics,filters
0,325286,12171,False,9031,Aksik,5.0,Все категории->Электроника->Смартфоны и телефо...,Зарядный кабель Borofone BX1 Lightning для айф...,"<p><span style=""background-color: transparent;...","[Длина: 1м, Разъем: Lightning, Подерживает быс...",{},"{'Цвет': ['Черный', 'Белый']}","{'Цвет': ['Белый', 'Черный']}"
1,888134,14233,False,18305,Sela,5.0,Все категории->Одежда->Женская одежда->Белье и...,Трусы Sela,Трусы-слипы из эластичного бесшовного трикотаж...,[],{},{'Размер одежды': ['S']},"{'Бренд': ['Sela'], 'Размер одежды': ['S']}"
3,1267173,13429,False,16357,ЮНЛАНДИЯ канцтовары,5.0,Все категории->Хобби и творчество->Рисование->...,"Гуашь ""ЮНЫЙ ВОЛШЕБНИК"", 12 цветов по 35 мл, БО...",<p>Гуашь высшего качества ЮНЛАНДИЯ поможет соз...,[],{},{},{'Бренд': ['Юнландия']}
4,1416943,2789,False,34666,вася-nicotine,4.0,Все категории->Хобби и творчество->Товары для ...,Колба для кальяна Крафт (разные цвета),"<p><span style=""color: rgb(149, 151, 153);"">Ун...","[Материал: стекло, Внутренний диаметр: 45 мм ,...",{},"{'Цвет': ['Хром', 'Розовый', 'Желтый', 'Прозра...","{'Цвет': ['Прозрачный', 'Желтый', 'Синий', 'Ро..."
5,1058275,12834,False,26389,Lim Market,4.6,Все категории->Одежда->Женская одежда->Домашня...,"Пижама женская, однотонная с шортами",<p>Лёгкая ткань! Комфортная посадка! Идеальная...,[],{},"{'Размер одежды': ['XXXL', 'L', 'XXL', 'XL'], ...","{'Цвет': ['Белый', 'Белый', 'Белый', 'Белый'],..."


Преобразуем столбец category_name в строку без спец символов

In [27]:
# final_train_data_copy = final_train_data.copy()

In [28]:
# final_train_data_copy['category_name'] = final_train_data_copy['category_name'].apply(lambda x: ' '.join(x.replace('Все категории->','').split('->')))

In [29]:
# final_train_data_copy['category_name'][0]

In [30]:
# final_train_data_copy.head(8)

Столбец с названием обрабатываем от знаков препинания

In [31]:
# final_train_data_copy['title'] = final_train_data['title'].apply(lambda x: re.sub(r'[^\w\s]','', x))

Столбец с описанием содержит теги html, удалим их. далее посмотрим что останется

In [32]:
# final_train_data_copy['description'] = final_train_data_copy['description'].apply(lambda x: ' '.join(re.findall(r'[А-Яа-я]+', x)))

Столбец с атрибутами содержит данные о размерах изделия

In [33]:
# final_train_data_copy['attributes'][0]        

In [34]:
# final_train_data_copy['attributes'] = final_train_data['attributes']

In [35]:
# final_train_data_copy['attributes'] = final_train_data_copy['attributes'].apply(lambda x: ' '.join(x).replace(': ', ' '))

In [36]:
# final_train_data_copy['attributes'].shape[0] - (final_train_data_copy['attributes'].loc[final_train_data_copy['attributes'] != '']).shape[0]

Пропусков < 1/3, заполним их словом Отсутствуют

In [37]:
# final_train_data_copy['attributes']

In [38]:
# for i in final_train_data_copy['attributes'].index:
#     if final_train_data_copy['attributes'][i] == '':
#         final_train_data_copy['attributes'][i] = 'Отсутствуют'
#     else:
#         continue

In [39]:
# final_train_data_copy['attributes']

Столбец custom_characteristics выглядит пустым, проверим

In [40]:
# final_train_data_copy['custom_characteristics'].loc[final_train_data_copy['custom_characteristics'] != {}]

In [41]:
# final_train_data_copy['custom_characteristics'][7]

Не будем использовать столбец custom_characteristics и filters, слишком много пропусков и данные разношерстные

In [42]:
# del final_train_data_copy['custom_characteristics']

In [43]:
# del final_train_data_copy['filters']

In [44]:
# final_train_data_copy.head()

Очистим столбец defined_characteristics

In [45]:
# final_train_data_copy['defined_characteristics'] = final_train_data['defined_characteristics'].apply(lambda x: re.sub(r'[^\w\s]','', str(x)))

In [46]:
# final_train_data_copy['defined_characteristics']

In [47]:
# final_train_data_copy['defined_characteristics'].shape[0] - (final_train_data_copy['defined_characteristics'].loc[final_train_data_copy['defined_characteristics'] != '']).shape[0]

Пропусков в столбике тоже больше 2/3. Удаляем

In [48]:
# del final_train_data_copy['defined_characteristics']

In [50]:
# final_train_data_copy.head()

NameError: ignored

In [76]:
def obrabotka(df):
    final_train_data_copy = df.copy()
    try:
        final_train_data_copy['category_name'] = final_train_data_copy['category_name'].apply(lambda x: ' '.join(x.replace('Все категории->','').split('->')))
    except:
        print('Takogo stolbca net!')
    finally:
        final_train_data_copy['title'] = final_train_data_copy['title'].apply(lambda x: re.sub(r'[^\w\s]','', x))
        final_train_data_copy['description'] = final_train_data_copy['description'].apply(lambda x: ' '.join(re.findall(r'[А-Яа-яеЕёЁ]+', x)))
        final_train_data_copy['attributes'] = final_train_data_copy['attributes'].apply(lambda x: ' '.join(x).replace(': ', ' '))
        
        for i in final_train_data_copy['attributes'].index:
            if final_train_data_copy['attributes'][i] == '':
                final_train_data_copy['attributes'][i] = 'Отсутствуют'
            else:
                continue

        del final_train_data_copy['custom_characteristics']
        del final_train_data_copy['filters']
        del final_train_data_copy['defined_characteristics']

    return final_train_data_copy

In [52]:
df_train = str_to_df(train_data)

In [53]:
df_train_final = obrabotka(df_train)

<ipython-input-51-765f82a05328>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_train_data_copy['attributes'][i] = 'Отсутствуют'


In [54]:
df_train_final.head()

,product_id,category_id,sale,shop_id,shop_title,rating,category_name,title,description,attributes
0,325286,12171,False,9031,Aksik,5.0,Электроника Смартфоны и телефоны Аксессуары и ...,Зарядный кабель Borofone BX1 Lightning для айф...,Зарядный кабель подходит для зарядки всех гадж...,Длина 1м Разъем Lightning Подерживает быструю ...
1,888134,14233,False,18305,Sela,5.0,Одежда Женская одежда Белье и купальники Трусы,Трусы Sela,Трусы слипы из эластичного бесшовного трикотаж...,Отсутствуют
3,1267173,13429,False,16357,ЮНЛАНДИЯ канцтовары,5.0,"Хобби и творчество Рисование Краски, пигменты",Гуашь ЮНЫЙ ВОЛШЕБНИК 12 цветов по 35 мл БОЛЬШИ...,Гуашь высшего качества ЮНЛАНДИЯ поможет создат...,Отсутствуют
4,1416943,2789,False,34666,вася-nicotine,4.0,Хобби и творчество Товары для курения Кальяны ...,Колба для кальяна Крафт разные цвета,Универсальная колба для кальяна подходит почти...,Материал стекло Внутренний диаметр 45 мм Высо...
5,1058275,12834,False,26389,Lim Market,4.6,Одежда Женская одежда Домашняя одежда Пижамы,Пижама женская однотонная с шортами,Лёгкая ткань Комфортная посадка Идеальная длин...,Отсутствуют


Теперь надо сделать обработчик тестовых данных.
Воспользуемся наработками

In [55]:
import requests
from bs4 import BeautifulSoup

In [56]:
test_data.head()

,product_id,sale,shop_id,shop_title,rating,text_fields
1,1997646,False,22758,Sky_Electronics,5.000000,"{""title"": ""Светодиодная лента Smart led Strip ..."
2,927375,False,17729,Di-Di Market,4.405941,"{""title"": ""Стекло ПЛЕНКА керамик матовое Honor..."
3,1921513,False,54327,VisionStore,4.000000,"{""title"": ""Проводные наушники с микрофоном jac..."
4,1668662,False,15000,FORNAILS,5.000000,"{""title"": ""Декоративная табличка \""Правила кух..."
5,1467778,False,39600,МОЯ КУХНЯ,5.000000,"{""title"": ""Подставка под ложку керамическая, п..."


In [57]:
id_product = test_data.product_id[1]
print(f'ИД товара для формирования запроса по API : {id_product}')

url = f'https://api.kazanexpress.ru/api/v2/product/{id_product}' # Формируем урл

request = requests.get(url)                         # GET Запрос
print(f'Статус запроса : {request}')                # Ответ

ИД товара для формирования запроса по API : 1997646
Статус запроса : <Response [200]>


In [58]:
print(request.text)

{"payload":{"data":{"id":1997646,"title":"Светодиодная лента Smart led Strip Light, с пультом, 5 метров, USB, Bluetooth","category":{"id":13495,"title":"Гирлянды","productAmount":2389,"parent":{"id":10508,"title":"Новогодние товары","productAmount":4711,"parent":{"id":10118,"title":"Товары для праздников","productAmount":24213,"parent":{"id":10018,"title":"Товары для дома","productAmount":67715,"parent":null}}}},"rating":5.0,"reviewsAmount":1,"ordersAmount":10,"rOrdersAmount":10,"totalAvailableAmount":13,"charityCommission":0,"description":"<p>Светодиодная лента LED, 5 м, RGB (Цветная) влагостойкая лента с пультом, USB адаптером, и возможностью управления с телефона.</p><p>Скачать приложение можно по ссылке <a href=\"http://www.qrtransfer.com/MiraclesStar.html\">http://www.qrtransfer.com/MiraclesStar.html</a></p><p>Гибкая, яркая, с хорошей световой отдачей и низкой ценой. Лента произведена на основе самоклеящейся печатной платы с прочным клеевым слоем «3М». Может нарезаться по 5 см (кр

In [59]:
# Сформируем json экземпляр, чтобы доставать информацию о продавце

res_req = json.loads(request.text)

In [60]:
def dict_to_lists(dct):
    """
    Функция для преобразования многоуровневого словаря в список с элементами ИД - категория
    """
    list_for_id = []
    list_for_title = []

    var_str = dct['payload']['data']['category'] #выбираем многоуровневый словарь с категориями
    var_str_1 = str(var_str).replace('{', '').replace("'", '').replace('}', '').replace(':', '') # Для простоты работаем с регулярками
    list_id = re.findall(r'id\s[0-9]+', var_str_1)
    
    for elem in list_id:
        list_for_id.append(int(elem.split(" ")[1]))

    list_title = re.findall(r'[а-яА-Я].*?[,]', var_str_1)

    for elem in list_title:
        list_for_title.append(elem.split(",")[0])

    result = list(zip(list_for_id, list_for_title))
    
    return result

In [61]:
var_ = dict_to_lists(res_req)

In [62]:
var_

[(13495, 'Гирлянды'),
 (10508, 'Новогодние товары'),
 (10118, 'Товары для праздников'),
 (10018, 'Товары для дома')]

In [63]:
list_cats = []

for elem in var_:
    list_cats.append(elem[1])

res_str = ' '.join(list_cats)

print(res_str)

Гирлянды Новогодние товары Товары для праздников Товары для дома


In [64]:
test_data_copy = test_data.copy()

In [72]:
train_data.loc[train_data['product_id'] == 927375]

,product_id,category_id,sale,shop_id,shop_title,rating,text_fields,category_name


In [65]:
test_data_copy.head()

,product_id,sale,shop_id,shop_title,rating,text_fields
1,1997646,False,22758,Sky_Electronics,5.000000,"{""title"": ""Светодиодная лента Smart led Strip ..."
2,927375,False,17729,Di-Di Market,4.405941,"{""title"": ""Стекло ПЛЕНКА керамик матовое Honor..."
3,1921513,False,54327,VisionStore,4.000000,"{""title"": ""Проводные наушники с микрофоном jac..."
4,1668662,False,15000,FORNAILS,5.000000,"{""title"": ""Декоративная табличка \""Правила кух..."
5,1467778,False,39600,МОЯ КУХНЯ,5.000000,"{""title"": ""Подставка под ложку керамическая, п..."


In [66]:
test_data_copy.loc[243
                   ]

product_id                                               1625690
sale                                                       False
shop_id                                                    45806
shop_title                            ООО «ТК Домашний Текстиль»
rating                                                  4.666667
text_fields    {"title": "Подушка \"Mia Cara\" wellness 70х70...
Name: 243, dtype: object

In [67]:
test_data_copy['category_name'] = 0

In [68]:
# list_cats = []
# list_all_cats = []

# for i in test_data_copy['product_id']:
#     id_product = i
#     url = f'https://api.kazanexpress.ru/api/v2/product/{id_product}' # Формируем урл
#     print(url)
#     request = requests.get(url)           # GET Запрос
#     print(f'Статус запроса : {request}')                # Ответ
#     res_req = json.loads(request.text)         
#     counter = 0
#     try:
#         var_ = dict_to_lists(res_req)
#     except:
#         counter += 1

#     for elem in var_:
#         list_cats.append(elem[1])


#     res_str = ' '.join(list_cats)
#     list_all_cats.extend(res_str)

In [69]:
# list_all_cats

NameError: ignored

In [73]:
final_test_data = str_to_df(test_data)

In [74]:
final_test_data.head()

,product_id,sale,shop_id,shop_title,rating,title,description,attributes,custom_characteristics,defined_characteristics,filters
1,1997646,False,22758,Sky_Electronics,5.000000,"Светодиодная лента Smart led Strip Light, с пу...","<p>Светодиодная лента LED, 5 м, RGB (Цветная) ...","[Легкость управления с пульта, а так же смартф...",{},{},"{'Тип питания': ['От сети', 'От USB'], 'В комп..."
2,927375,False,17729,Di-Di Market,4.405941,Стекло ПЛЕНКА керамик матовое Honor 50 lite 10...,"<p><span style=""color: rgb(63, 62, 62);"">Защит...",[Honor 50 ПЛЕНКА!!! КРАЯ КЛЕЮТСЯ МОГУТ НЕ ПРО...,"{'Модель': ['8a - Y6 2019', '9x - P Smart Z', ...",{},"{'Модель устройства': ['Honor 9X', 'Honor 20 P..."
3,1921513,False,54327,VisionStore,4.000000,"Проводные наушники с микрофоном jack 3.5, IOS,...",<p>Наушники проводные с микрофоном отличное ка...,[Возможность использования как гарнитуры для П...,{},{},"{'Разъем': ['3.5 мм'], 'Конструкция наушников'..."
4,1668662,False,15000,FORNAILS,5.000000,"Декоративная табличка ""Правила кухни"", подстав...","<p>Декоративная табличка ""Правила кухни"" созда...",[],{},{},{'Бренд': ['FORNAILS']}
5,1467778,False,39600,МОЯ КУХНЯ,5.000000,"Подставка под ложку керамическая, подложка ""Кл...",<p>Подложка керамическая с рисунком в подарочн...,"[Керамика., Размер: 255*90*30мм; ]","{'Модель': ['Клубника', 'Лимоны', 'Лаванда']}",{},"{'Бренд': ['Valleya', 'Valleya', 'Valleya']}"


In [77]:
ffinal_test = obrabotka(final_test_data)

Takogo stolbca net!


<ipython-input-76-144c6fcdf7c8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_train_data_copy['attributes'][i] = 'Отсутствуют'


In [78]:
ffinal_test.head()

,product_id,sale,shop_id,shop_title,rating,title,description,attributes
1,1997646,False,22758,Sky_Electronics,5.000000,Светодиодная лента Smart led Strip Light с пул...,Светодиодная лента м Цветная влагостойкая лент...,"Легкость управления с пульта, а так же смартфо..."
2,927375,False,17729,Di-Di Market,4.405941,Стекло ПЛЕНКА керамик матовое Honor 50 lite 10...,Защитное матовое керамическое стекло эффективн...,Honor 50 ПЛЕНКА!!! КРАЯ КЛЕЮТСЯ МОГУТ НЕ ПРОК...
3,1921513,False,54327,VisionStore,4.000000,Проводные наушники с микрофоном jack 35 IOS An...,Наушники проводные с микрофоном отличное качес...,Возможность использования как гарнитуры для ПК...
4,1668662,False,15000,FORNAILS,5.000000,Декоративная табличка Правила кухни подставка ...,Декоративная табличка Правила кухни создаст не...,Отсутствуют
5,1467778,False,39600,МОЯ КУХНЯ,5.000000,Подставка под ложку керамическая подложка Клуб...,Подложка керамическая с рисунком в подарочной ...,Керамика. Размер 255*90*30мм;


In [80]:
df_train_final.head()

,product_id,category_id,sale,shop_id,shop_title,rating,category_name,title,description,attributes
0,325286,12171,False,9031,Aksik,5.0,Электроника Смартфоны и телефоны Аксессуары и ...,Зарядный кабель Borofone BX1 Lightning для айф...,Зарядный кабель подходит для зарядки всех гадж...,Длина 1м Разъем Lightning Подерживает быструю ...
1,888134,14233,False,18305,Sela,5.0,Одежда Женская одежда Белье и купальники Трусы,Трусы Sela,Трусы слипы из эластичного бесшовного трикотаж...,Отсутствуют
3,1267173,13429,False,16357,ЮНЛАНДИЯ канцтовары,5.0,"Хобби и творчество Рисование Краски, пигменты",Гуашь ЮНЫЙ ВОЛШЕБНИК 12 цветов по 35 мл БОЛЬШИ...,Гуашь высшего качества ЮНЛАНДИЯ поможет создат...,Отсутствуют
4,1416943,2789,False,34666,вася-nicotine,4.0,Хобби и творчество Товары для курения Кальяны ...,Колба для кальяна Крафт разные цвета,Универсальная колба для кальяна подходит почти...,Материал стекло Внутренний диаметр 45 мм Высо...
5,1058275,12834,False,26389,Lim Market,4.6,Одежда Женская одежда Домашняя одежда Пижамы,Пижама женская однотонная с шортами,Лёгкая ткань Комфортная посадка Идеальная длин...,Отсутствуют


ПРобуем NLP

In [83]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [94]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [93]:
text = df_train_final['title'][5]
word_tokenize(text, language="russian")

['Пижама', 'женская', 'однотонная', 'с', 'шортами']